# SAY SOSMETHING HERE


 Your task will be to help a company operating small and mid-size apartments hosting 2-6 guests. The company is set to price their new apartments not on the market.
 
- Build a price prediction model similarly to how we did in our case study for London.
- Discuss your modeling decisions and compare your results to those of the case study.

In [1]:
#@ least 3 different models: (I have to argue for my choice of models)

#Model 1: OLS Linear Regression
#Model 2: LASSO 
#Model 3: Regression Tree (CART)
#Model 4: Random Forest(RF)

### Importing Libraries

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from plotnine import *

### Reading Data & Exploratory Data Analysis

In [4]:
data = pd.read_csv('austinairbnblistings.csv')

In [5]:
data.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5456,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,8028,Sylvia,NaN,78702,30.26057,-97.73441,Entire home/apt,101.0,2,668,2023-11-20,3.71,1,330,47,NaN
1,5769,Home in Austin · ★4.91 · 1 bedroom · 1 bed · 1...,8186,Elizabeth,NaN,78729,30.45697,-97.78422,Private room,NaN,1,294,2023-12-07,1.76,1,0,20,NaN
2,218402,Home in Austin · ★4.74 · 1 bedroom · 1 bed · 1...,1129520,Paulette,NaN,78754,30.36145,-97.64397,Private room,30.0,28,29,2023-09-24,0.20,1,252,2,NaN


In [6]:
data.shape

(15419, 18)

In [7]:
data = data.loc[data['room_type'] == 'Entire home/apt']

In [8]:
in_depth_data = pd.read_csv('indepthaustinairbnb.csv')

In [9]:
in_depth_data.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5456,https://www.airbnb.com/rooms/5456,20231215200307,2023-12-16,city scrape,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,NaN,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,...,4.83,4.73,4.79,NaN,f,1,1,0,0,3.71
1,5769,https://www.airbnb.com/rooms/5769,20231215200307,2023-12-16,previous scrape,Home in Austin · ★4.91 · 1 bedroom · 1 bed · 1...,NaN,Quiet neighborhood with lots of trees and good...,https://a0.muscache.com/pictures/23822033/ac94...,8186,...,4.94,4.76,4.92,NaN,f,1,0,1,0,1.76
2,6413,https://www.airbnb.com/rooms/6413,20231215200307,2023-12-16,previous scrape,Guesthouse in Austin · ★4.97 · Studio · 1 bed ...,NaN,Travis Heights is one of the oldest neighborho...,https://a0.muscache.com/pictures/miso/Hosting-...,13879,...,4.98,4.87,4.93,NaN,f,1,1,0,0,0.70


In [10]:
in_depth_data = in_depth_data.loc[(in_depth_data['accommodates'] >=2) & (in_depth_data['accommodates'] <=6) ]

In [11]:
in_depth_data.accommodates.value_counts()

accommodates
2    3964
4    3340
6    2293
3    1047
5     837
Name: count, dtype: int64

In [12]:
#created new df airbnb (combines both previous data frames)
airbnb = pd.merge(data, in_depth_data, on = 'id')

In [13]:
airbnb.head(3)

,id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude_x,longitude_x,room_type_x,price_x,...,review_scores_communication,review_scores_location,review_scores_value,license_y,instant_bookable,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y
0,5456,Guesthouse in Austin · ★4.84 · 1 bedroom · 2 b...,8028,Sylvia,NaN,78702,30.26057,-97.73441,Entire home/apt,101.0,...,4.83,4.73,4.79,NaN,f,1,1,0,0,3.71
1,6413,Guesthouse in Austin · ★4.97 · Studio · 1 bed ...,13879,Todd,NaN,78704,30.24885,-97.73587,Entire home/apt,NaN,...,4.98,4.87,4.93,NaN,f,1,1,0,0,0.70
2,6448,Guesthouse in Austin · ★4.97 · 1 bedroom · 2 b...,14156,Amy,NaN,78704,30.26034,-97.76487,Entire home/apt,159.0,...,4.97,4.97,4.89,NaN,t,1,1,0,0,2.09


In [14]:
airbnb.rename(columns = {'price_x':'price'}, inplace = True)

In [15]:
airbnb.columns

Index(['id', 'name_x', 'host_id_x', 'host_name_x', 'neighbourhood_group',
       'neighbourhood_x', 'latitude_x', 'longitude_x', 'room_type_x', 'price',
       'minimum_nights_x', 'number_of_reviews_x', 'last_review_x',
       'reviews_per_month_x', 'calculated_host_listings_count_x',
       'availability_365_x', 'number_of_reviews_ltm_x', 'license_x',
       'listing_url', 'scrape_id', 'last_scraped', 'source', 'name_y',
       'description', 'neighborhood_overview', 'picture_url', 'host_id_y',
       'host_url', 'host_name_y', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_y',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude_y',
       'lo

In [16]:
airbnb['bathrooms'] = airbnb['bathrooms_text'].apply(lambda x: float(x.split()[0]) if x.split()[0].replace('.', '').isdigit() else None)
#turned observations into floats 

In [17]:
airbnb['host_is_superhost'] = airbnb['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)
#turned observations into booleans

In [18]:
airbnb.maximum_nights

0         90
1         90
2        365
3         60
4         90
        ... 
9343    1125
9344     365
9345     365
9346     365
9347     365
Name: maximum_nights, Length: 9348, dtype: int64

In [19]:
#ONLY FOR OLS LINEAR REGRESSION (doing variable selection and interaction selection only for linear regression)
#my other models will have variables chosen on their own due to algorithms!!!
# y = price
# x = bathrooms, beds, review_scores_rating (would have to group), property_type, availability_365_x???, 
#number_of_reviews???, host_acceptance_rate??????, host_is_superhost, minimum_nights_y, maximum_nights,
# interaction terms: (before doing these for sure, do scatterplot to see their correlation) bathrooms*beds, 
#host_acceptance_rate*host_is_superhost, maximum_nights*minimum_nights, availability*number_of_reviews 

In [77]:
#must drop any null values in outcome variable (price)
airbnb.price.isnull().sum()
airbnb.dropna(subset = ['price'], inplace = True)

#bathroom observations already turned into floats. now dropping null bathrooms (only 3)
airbnb.bathrooms.isnull().sum()
airbnb.dropna(subset = ['bathrooms'], inplace = True)

#beds. dropping null beds (only 43)
airbnb.beds.isnull().sum()
airbnb.dropna(subset = ['beds'], inplace = True)

#review_scores_rating. over 1000 null values. will impute with median as distribution looks far 
#from normal (used ggplot to plot distribution). will also create flagged variable for missing review_scores_rating
#and flag variable for imputed review_scores_rating
airbnb.review_scores_rating.isnull().sum()
airbnb['missing_review_scores_flag'] = airbnb.review_scores_rating.isnull().astype(int)
airbnb['review_scores_imputed_flag'] = airbnb.review_scores_rating.isnull().astype(int)
airbnb.fillna({'review_scores_rating': airbnb['review_scores_rating'].median()}, inplace=True)

In [78]:
airbnb.loc[airbnb['review_scores_rating'] == airbnb['review_scores_rating'].median()]

,id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude_x,longitude_x,room_type_x,price,...,review_scores_value,license_y,instant_bookable,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y,missing_review_scores_flag,review_scores_imputed_flag
9,233930,Home in Austin · ★4.93 · 2 bedrooms · 2 beds ·...,578211,Pete,NaN,78751,30.302890,-97.729610,Entire home/apt,114.0,...,4.79,NaN,f,10,10,0,0,0.73,0,0
42,463790,Home in Austin · 3 bedrooms · 3 beds · 3 baths,1487463,Al,NaN,78736,30.239450,-97.903950,Entire home/apt,125.0,...,NaN,NaN,f,1,1,0,0,NaN,0,0
59,510168,Home in Austin · ★4.93 · 3 bedrooms · 7 beds ·...,793021,Steve,NaN,78733,30.342530,-97.861600,Entire home/apt,325.0,...,4.85,NaN,t,1,1,0,0,1.13,0,0
92,345536,Bungalow in Austin · ★4.93 · 2 bedrooms · 3 be...,1752493,Gigi,NaN,78704,30.246600,-97.761570,Entire home/apt,170.0,...,4.85,NaN,t,2,2,0,0,2.25,0,0
96,622495,Condo in Austin · ★4.93 · 1 bedroom · 2 beds ·...,12409,Flip,NaN,78705,30.287140,-97.752580,Entire home/apt,81.0,...,4.76,NaN,f,2,2,0,0,0.31,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9342,1038772488249130809,Rental unit in Austin · ★New · 2 bedrooms · 3 ...,141885946,CozySuites Austin,NaN,78745,30.212351,-97.772822,Entire home/apt,79.0,...,NaN,NaN,t,16,16,0,0,NaN,0,0
9344,1039120311726737748,Tiny home in Austin · ★New · 1 bedroom · 1 bed...,206537515,Mo,NaN,78721,30.282640,-97.692550,Entire home/apt,150.0,...,NaN,NaN,f,1,1,0,0,NaN,0,0
9345,1046446724556255368,Rental unit in Austin · ★New · 1 bedroom · 1 b...,545892202,Monthier,NaN,78701,30.261865,-97.742385,Entire home/apt,92.0,...,NaN,NaN,t,4,4,0,0,NaN,0,0
9346,1039276072771109547,Home in Austin · ★New · 3 bedrooms · 3 beds · ...,7363313,Macy,NaN,78754,30.360289,-97.663187,Entire home/apt,175.0,...,NaN,NaN,f,11,7,4,0,NaN,0,0
